In [8]:
import os
import IPython
from pydub import AudioSegment
from utils import Datainfo

In [9]:
posDir = Datainfo["positives"]
negDir = Datainfo["negatives"]
templates = Datainfo["backgrounds"]

In [13]:
# Unit test

sampleDir = os.path.join(posDir, os.listdir(posDir)[0])
audioSample = AudioSegment.from_wav(sampleDir)

templateDir = os.path.join(templates, os.listdir(templates)[0])
template = AudioSegment.from_wav(templateDir)

In [15]:
test = template.overlay(audioSample, position=1000)

In [18]:
test.export(os.path.join(Datainfo["dir"], "tests", "overlayTest.wav"), format="wav")

<_io.BufferedRandom name='e:\\Prototype\\WWmodule\\Data\\tests\\overlayTest.wav'>